####### ref : https://docs.databricks.com/aws/en/data-governance/unity-catalog/

# Volumes

In [0]:
dbutils.fs.ls('abfss://gatovolume@gatouswestorage20250811.dfs.core.windows.net/')

[FileInfo(path='abfss://gatovolume@gatouswestorage20250811.dfs.core.windows.net/RAW/', name='RAW/', size=0, modificationTime=1755789750000)]

In [0]:
dbutils.fs.mkdirs('abfss://gatovolume@gatouswestorage20250811.dfs.core.windows.net/ExternalSink/')

True

In [0]:
dbutils.fs.ls('abfss://gatovolume@gatouswestorage20250811.dfs.core.windows.net/')

[FileInfo(path='abfss://gatovolume@gatouswestorage20250811.dfs.core.windows.net/ExternalSink/', name='ExternalSink/', size=0, modificationTime=1755793396000),
 FileInfo(path='abfss://gatovolume@gatouswestorage20250811.dfs.core.windows.net/RAW/', name='RAW/', size=0, modificationTime=1755789750000)]

In [0]:
_catalog = spark.catalog
_catalogs = _catalog.listCatalogs()
for cat in _catalogs:
	print(f"Catalog Name: {cat.name}")

Catalog Name: autoloadcatalog
Catalog Name: catalogo_gatuno
Catalog Name: catalogo_gatuno_externo
Catalog Name: hive_metastore
Catalog Name: main
Catalog Name: samples
Catalog Name: system


###External Catalog
*don't forget to create the unity catalog **_BERFORE_** the compute cluster*

In [0]:
%sql
CREATE CATALOG catalogo_gatuno_volumenes
MANAGED LOCATION 'abfss://gatovolume@gatouswestorage20250811.dfs.core.windows.net/ExternalCatalogs';

In [0]:
%sql
CREATE SCHEMA catalogo_gatuno_volumenes.esquema_gatuno_externo;

In [0]:
#
## REF : https://docs.databricks.com/aws/en/sql/language-manual/sql-ref-volumes
#
_query = "CREATE EXTERNAL VOLUME IF NOT EXISTS catalogo_gatuno_volumenes.esquema_gatuno_externo.MiVolumenGatuno COMMENT 'This is my example external volume' LOCATION 'abfss://gatovolume@gatouswestorage20250811.dfs.core.windows.net/ExternalSink/'"
#
spark.sql(sqlQuery=_query)
#

DataFrame[]

##### https://github.com/toUpperCase78/formula1-datasets/blob/master/Formula1_2024season_drivers.csv

In [0]:
%fs
ls 'abfss://gatovolume@gatouswestorage20250811.dfs.core.windows.net/ExternalSink/'

path,name,size,modificationTime
abfss://gatovolume@gatouswestorage20250811.dfs.core.windows.net/ExternalSink/Formula1_2024season_drivers.csv,Formula1_2024season_drivers.csv,2388,1755794790000


In [0]:
%fs
ls '/Volumes/catalogo_gatuno_volumenes/esquema_gatuno_externo/mivolumengatuno'

path,name,size,modificationTime
dbfs:/Volumes/catalogo_gatuno_volumenes/esquema_gatuno_externo/mivolumengatuno/Formula1_2024season_drivers.csv,Formula1_2024season_drivers.csv,2388,1755794790000


In [0]:
from pyspark.sql.functions import col
#
display(spark.read \
	.option('header', True) \
	.option('inferSchema', True) \
	.csv('/Volumes/catalogo_gatuno_volumenes/esquema_gatuno_externo/mivolumengatuno/Formula1_2024season_drivers.csv') \
	.where(col('No') != '4') \
	.filter(~col('No').isin(([4,63]))) \
	.limit(5))

Driver,Abbreviation,No,Team,Country,Podiums,Points,GrandsPrixEntered,WorldChampionships,HighestRaceFinish,HighestGridPosition,DateOfBirth,PlaceOfBirth
Max Verstappen,VER,1,Red Bull Racing,Netherlands,112,3023.5,209,4,1(x63),1,1997-09-30,"Hasselt, Belgium"
Charles Leclerc,LEC,16,Ferrari,Monaco,43,1430.0,149,0,1(x8),1,1997-10-16,"Monte Carlo, Monaco"
Oscar Piastri,PIA,81,McLaren,Australia,10,389.0,46,0,1(x2),2,2001-04-06,"Melbourne, Victoria"
Carlos Sainz,SAI,55,Ferrari,Spain,27,1272.5,208,0,1(x4),1,1994-09-01,"Madrid, Spain"
Lewis Hamilton,HAM,44,Mercedes,United Kingdom,202,4862.5,356,7,1(x105),1,1985-01-07,"Stevenage, England"


In [0]:
%python
with open('/Volumes/catalogo_gatuno_volumenes/esquema_gatuno_externo/mivolumengatuno/Formula1_2024season_drivers.csv', 'r') as f:
	print(f.readline())

Driver,Abbreviation,No,Team,Country,Podiums,Points,GrandsPrixEntered,WorldChampionships,HighestRaceFinish,HighestGridPosition,DateOfBirth,PlaceOfBirth



In [0]:
((spark.readStream \
		.format('cloudFiles') \
		.option('cloudFiles.format', 'csv') \
		.option('cloudFiles.schemaLocation','abfss://gatovolume@gatouswestorage20250811.dfs.core.windows.net/Schemas/') \
		.option('header', True) \
		.option('inferSchema', True) \
		.load('/Volumes/catalogo_gatuno_volumenes/esquema_gatuno_externo/mivolumengatuno/')) \
	.writeStream.format('delta') \
		.option('checkpointLocation','abfss://gatovolume@gatouswestorage20250811.dfs.core.windows.net/Checkpoint/') \
		.outputMode('append') \
		.trigger(once=True) \
		.toTable('catalogo_gatuno_volumenes.esquema_gatuno_externo.FormulaUnoDrivers2024'))

In [0]:
%sql
SELECT
	*
FROM
	catalogo_gatuno_volumenes.esquema_gatuno_externo.FormulaUnoDrivers2024
WHERE
	No NOT IN (4,63)
LIMIT 5;

Driver,Abbreviation,No,Team,Country,Podiums,Points,GrandsPrixEntered,WorldChampionships,HighestRaceFinish,HighestGridPosition,DateOfBirth,PlaceOfBirth,_rescued_data
Max Verstappen,VER,1,Red Bull Racing,Netherlands,112,3023.5,209,4,1(x63),1,30/09/1997,"Hasselt, Belgium",null
Charles Leclerc,LEC,16,Ferrari,Monaco,43,1430,149,0,1(x8),1,16/10/1997,"Monte Carlo, Monaco",null
Oscar Piastri,PIA,81,McLaren,Australia,10,389,46,0,1(x2),2,06/04/2001,"Melbourne, Victoria",null
Carlos Sainz,SAI,55,Ferrari,Spain,27,1272.5,208,0,1(x4),1,01/09/1994,"Madrid, Spain",null
Lewis Hamilton,HAM,44,Mercedes,United Kingdom,202,4862.5,356,7,1(x105),1,07/01/1985,"Stevenage, England",null


In [0]:
%sql
SELECT
	*
FROM
	csv.`/Volumes/catalogo_gatuno_volumenes/esquema_gatuno_externo/mivolumengatuno/`
WHERE
	_c3 NOT IN (4, 63)
LIMIT 2;

_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10,_c11,_c12
Driver,Abbreviation,No,Team,Country,Podiums,Points,GrandsPrixEntered,WorldChampionships,HighestRaceFinish,HighestGridPosition,DateOfBirth,PlaceOfBirth
Max Verstappen,VER,1,Red Bull Racing,Netherlands,112,3023.5,209,4,1(x63),1,30/09/1997,"Hasselt, Belgium"


In [0]:
%sql
LIST '/Volumes/catalogo_gatuno_volumenes/esquema_gatuno_externo/mivolumengatuno/'

path,name,size,modification_time
/Volumes/catalogo_gatuno_volumenes/esquema_gatuno_externo/mivolumengatuno/Formula1_2024season_drivers.csv,Formula1_2024season_drivers.csv,2377,1755805491000


In [0]:
%sql
SELECT
	*
FROM
	csv.`dbfs:/Volumes/catalogo_gatuno_volumenes/esquema_gatuno_externo/mivolumengatuno/`
WHERE
	_c3 NOT IN (4, 63)
LIMIT 2

_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10,_c11,_c12
Driver,Abbreviation,No,Team,Country,Podiums,Points,GrandsPrixEntered,WorldChampionships,HighestRaceFinish,HighestGridPosition,DateOfBirth,PlaceOfBirth
Max Verstappen,VER,1,Red Bull Racing,Netherlands,112,3023.5,209,4,1(x63),1,30/09/1997,"Hasselt, Belgium"
